# Data Cleaning

In [1]:
import pandas as pd
actor_df = pd.read_csv('actors.csv')

actor_df

,id,name,role
0,1000001,Margot Robbie,Barbie
1,1000001,Ryan Gosling,Ken
2,1000001,America Ferrera,Gloria
3,1000001,Ariana Greenblatt,Sasha
4,1000001,Issa Rae,Barbie
...,...,...,...
5798445,1941596,Marc Ma,Ba Cai/巴莱
5798446,1941596,线雨轩,Tata/塔塔
5798447,1941596,Jiang Yixuan,Zuo Yila（Zoila）/佐伊拉
5798448,1941597,Hiroshi Mikami,NaN


In [2]:
import pandas as pd
actor_df = pd.read_csv('actors.csv')
actor_cleaned = actor_df[(actor_df['role'].notnull()) & (actor_df['role'] != '')]

actor_cleaned

,id,name,role
0,1000001,Margot Robbie,Barbie
1,1000001,Ryan Gosling,Ken
2,1000001,America Ferrera,Gloria
3,1000001,Ariana Greenblatt,Sasha
4,1000001,Issa Rae,Barbie
...,...,...,...
5798442,1941596,Nick Cheung,Zhang Yao/张耀
5798443,1941596,Ethan Juan,An Du/安渡
5798445,1941596,Marc Ma,Ba Cai/巴莱
5798446,1941596,线雨轩,Tata/塔塔
